In [1]:
import pandas as pd
import numpy as np

In [2]:
#load data

In [3]:
daily_df = pd.read_csv("dataset/stations_daily.csv")

In [4]:
hourly_df = pd.read_csv("dataset/stations_hourly.csv")

In [5]:
rsinaica_df = pd.read_csv("dataset/stations_rsinaica.csv")

# จัดการกับข้อมูลรายงานฝุ่นรายวันก่อน

we will use this dataframe to calculate the yearly statistic of ฝุ่น

In [6]:
daily_df.head()

,datetime,station_id,PM2.5,PM10,NOx,O3,CO,HR,NO,NO2,...,HCT,HRI,IUV,PB,PP,PST,RS,TMPI,UVA,XIL
0,2015-04-02,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.361000,NaN,NaN
1,2015-05-21,32,0.454967,0.347400,0.059194,0.029176,NaN,44.879091,0.029525,0.053121,...,NaN,NaN,NaN,570.775455,0.296364,NaN,7.871636,16.792818,NaN,NaN
2,2015-05-22,32,0.077821,0.389323,0.019158,0.051903,NaN,42.506000,0.009589,0.014215,...,NaN,NaN,NaN,558.360000,0.002750,NaN,384.788079,17.018949,NaN,NaN
3,2015-05-23,32,0.018036,0.387729,0.019711,0.021092,0.144603,54.734437,0.009863,0.013436,...,NaN,NaN,NaN,573.282187,0.038750,NaN,252.605975,18.575000,NaN,NaN
4,2015-05-24,32,0.024935,0.398042,0.024757,0.009568,0.494143,58.374574,0.012397,0.010337,...,NaN,NaN,NaN,572.169574,0.003404,NaN,259.231664,16.781064,NaN,NaN


In [7]:
daily_df.shape

(231592, 27)

Let's add year and month columns to daily_df 

In [8]:
daily_df.insert(1, "year", daily_df["datetime"].apply(lambda x: x.split("-")[0]))
daily_df.insert(1, "month", daily_df["datetime"].apply(lambda x: x.split("-")[1]))

convert month and year to int \
then sort values by month and year

In [9]:
daily_df["month"] = pd.to_numeric(daily_df["month"])
daily_df["year"] = pd.to_numeric(daily_df["year"])

daily_df.sort_values(by=["year", "month"], inplace = True)
daily_df.head()

,datetime,month,year,station_id,PM2.5,PM10,NOx,O3,CO,HR,...,HCT,HRI,IUV,PB,PP,PST,RS,TMPI,UVA,XIL
59820,2000-01-01,1,2000,102,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59821,2000-01-02,1,2000,102,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59822,2000-01-03,1,2000,102,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59823,2000-01-04,1,2000,102,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59824,2000-01-05,1,2000,102,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


count the values exist each year

In [10]:
groupDaily_by_year = daily_df.groupby("year")
groupDaily_by_year.count()
# we will keep a year if PM2.5 have data > 30 %

def keep_year(group):
    return ((group.count()["PM2.5"] * 100) / len(groupDaily_by_year) > 30)
groupDaily_by_year.count()

,datetime,month,station_id,PM2.5,PM10,NOx,O3,CO,HR,NO,...,HCT,HRI,IUV,PB,PP,PST,RS,TMPI,UVA,XIL
year,,,,,,,,,,,,,,,,,,,,,
2000,1678,1678,1678,0,0,0,0,0,0,1678,...,0,0,0,0,0,0,0,0,0,0
2001,1658,1658,1658,0,0,0,0,0,0,1658,...,0,0,0,0,0,0,0,0,0,0
2002,826,826,826,0,0,0,0,0,0,826,...,0,0,0,0,0,0,0,0,0,0
2006,321,321,321,0,0,0,0,0,0,321,...,0,0,0,0,0,0,0,0,0,0
2007,44,44,44,0,0,0,0,0,0,44,...,0,0,0,0,0,0,0,0,0,0
2008,43,43,43,0,0,0,0,0,0,43,...,0,0,0,0,0,0,0,0,0,0
2009,43,43,43,0,0,0,0,0,0,43,...,0,0,0,0,0,0,0,0,0,0
2010,10417,10417,10417,3814,7612,151,8235,6816,151,1069,...,0,0,0,140,0,0,0,0,0,0
2011,11315,11315,11315,5374,8473,61,10057,6298,65,1836,...,0,29,0,36,27,0,0,25,0,0


filter data

In [11]:
daily_df = groupDaily_by_year.filter(keep_year)
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226979 entries, 12133 to 231591
Data columns (total 29 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   datetime    226979 non-null  object 
 1   month       226979 non-null  int64  
 2   year        226979 non-null  int64  
 3   station_id  226979 non-null  int64  
 4   PM2.5       161698 non-null  float64
 5   PM10        180481 non-null  float64
 6   NOx         112881 non-null  float64
 7   O3          192957 non-null  float64
 8   CO          175493 non-null  float64
 9   HR          123285 non-null  float64
 10  NO          113541 non-null  float64
 11  NO2         169286 non-null  float64
 12  TMP         127015 non-null  float64
 13  BEN         775 non-null     float64
 14  CH4         270 non-null     float64
 15  CN          595 non-null     float64
 16  CO2         56 non-null      float64
 17  H2S         280 non-null     float64
 18  HCNM        279 non-null     float64
 19

next we will drop column which have few data

In [12]:
daily_df.isna().sum()* 100 / len(daily_df)

#show % of Nan values in each column, we will drop at 51 %

datetime        0.000000
month           0.000000
year            0.000000
station_id      0.000000
PM2.5          28.760810
PM10           20.485596
NOx            50.268086
O3             14.989052
CO             22.683156
HR             45.684403
NO             49.977311
NO2            25.417770
TMP            44.041079
BEN            99.658559
CH4            99.881046
CN             99.737861
CO2            99.975328
H2S            99.876641
HCNM           99.877081
HCT           100.000000
HRI            95.503989
IUV            97.825349
PB             59.583045
PP             66.804859
PST           100.000000
RS             66.733927
TMPI           84.851021
UVA            92.911238
XIL            99.658559
dtype: float64

Too many Nan values OMG

In [14]:
for col in daily_df:
    if daily_df[col].isna().sum()* 100 / len(daily_df) > 51:
        daily_df.drop(col, axis=1, inplace=True)

daily_df

,datetime,month,year,station_id,PM2.5,PM10,NOx,O3,CO,HR,NO,NO2,TMP
12133,2010-01-20,1,2010,68,NaN,50.444444,0.019147,NaN,NaN,35.205704,0.007604,0.011870,22.005545
12134,2010-01-21,1,2010,68,NaN,67.500000,0.042956,NaN,NaN,42.276420,0.028849,0.014479,16.882448
12135,2010-01-22,1,2010,68,NaN,67.291667,NaN,NaN,NaN,NaN,NaN,0.015071,NaN
12136,2010-01-23,1,2010,68,NaN,69.500000,NaN,NaN,NaN,NaN,NaN,0.014679,NaN
12137,2010-01-24,1,2010,68,NaN,47.500000,NaN,NaN,NaN,NaN,NaN,0.013279,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
231587,2021-12-27,12,2021,426,28.956522,67.375000,0.036429,0.030750,1.684583,56.541667,NaN,0.024762,21.660833
231588,2021-12-28,12,2021,426,20.818182,55.833333,0.037508,0.026792,1.700000,44.375000,NaN,0.023225,23.731250
231589,2021-12-29,12,2021,426,16.857143,51.458333,0.045588,0.017083,1.732917,37.875000,NaN,0.026500,22.751250
231590,2021-12-30,12,2021,426,21.555556,51.750000,0.037229,0.029833,1.520000,42.916667,NaN,0.023929,22.988333


# hourly_df

In [15]:
hourly_df.head()

,datetime,station_id,PM2.5,PM10,NOx,O3,CO,HR,NO,NO2,...,HCT,HRI,IUV,PB,PP,PST,RS,TMPI,UVA,XIL
0,2015-04-02 09:00:00,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.361,NaN,NaN
1,2015-05-21 18:00:00,32,0.97112,NaN,NaN,NaN,NaN,24.416,NaN,NaN,...,NaN,NaN,NaN,569.61,0.00,NaN,41.1090,16.723,NaN,NaN
2,2015-05-21 19:00:00,32,0.97122,0.18008,0.111060,0.020706,NaN,31.282,0.005918,0.104480,...,NaN,NaN,NaN,570.01,1.55,NaN,8.0007,16.745,NaN,NaN
3,2015-05-21 19:00:00,32,0.97122,0.18008,0.111060,0.020706,NaN,31.282,0.104480,0.104480,...,NaN,NaN,NaN,570.01,1.55,NaN,8.0007,16.745,NaN,NaN
4,2015-05-21 20:00:00,32,0.93155,0.38415,0.058019,0.030228,NaN,36.145,0.003471,0.054585,...,NaN,NaN,NaN,569.97,0.00,NaN,7.3412,16.586,NaN,NaN


# rsinaica_df

In [16]:
hourly_df.drop_duplicates(["datetime", "station_id"])
hourly_df.sort_values("datetime")

,datetime,station_id,PM2.5,PM10,NOx,O3,CO,HR,NO,NO2,...,HCT,HRI,IUV,PB,PP,PST,RS,TMPI,UVA,XIL
1838329,2000-01-01 00:00:00,109,NaN,NaN,NaN,NaN,NaN,NaN,0.039,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2771691,2000-01-01 00:00:00,128,NaN,NaN,NaN,NaN,NaN,NaN,0.010,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1474466,2000-01-01 00:00:00,102,NaN,NaN,NaN,NaN,NaN,NaN,0.019,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4652322,2000-01-01 00:00:00,256,NaN,NaN,NaN,NaN,NaN,NaN,0.027,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4832607,2000-01-01 00:00:00,260,NaN,NaN,NaN,NaN,NaN,NaN,0.061,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4652320,2021-12-31 23:00:00,256,33.00,59.000,0.06800,0.0020,1.60,48.0,NaN,0.05300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1838327,2021-12-31 23:00:00,109,NaN,47.873,NaN,0.0194,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.053,NaN,NaN
3540694,2021-12-31 23:00:00,143,NaN,154.000,0.10920,0.0040,2.73,60.0,NaN,0.05450,...,NaN,NaN,NaN,715.7,0.0,NaN,0.0,NaN,NaN,NaN
443186,2021-12-31 23:00:00,70,26.09,65.470,0.01515,0.0126,2.19,NaN,NaN,0.00808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
rsinaica_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341 entries, 0 to 340
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   station_id         341 non-null    int64  
 1   station_name       341 non-null    object 
 2   station_code       341 non-null    object 
 3   network_id         341 non-null    int64  
 4   network_name       334 non-null    object 
 5   network_code       340 non-null    object 
 6   street             187 non-null    object 
 7   ext                154 non-null    object 
 8   interior           74 non-null     object 
 9   colonia            186 non-null    object 
 10  zip                143 non-null    object 
 11  state_code         299 non-null    float64
 12  municipio_code     299 non-null    float64
 13  year_started       138 non-null    float64
 14  altitude           320 non-null    float64
 15  address            138 non-null    object 
 16  date_validated     20 non-

make new dataframe caontain station_id and network_name(city)

In [18]:
data = {"station_id":rsinaica_df["station_id"],
        "network_name":rsinaica_df["network_name"]}
new_df = pd.DataFrame(data)

In [19]:
new_df.fillna("Unknown")

,station_id,network_name
0,397,Unknown
1,344,Unknown
2,365,Unknown
3,393,Unknown
4,394,Unknown
...,...,...
336,239,Zacatecas
337,295,Zacatecas
338,238,Zacatecas
339,135,Zacatepec


#  prepare data for คำนวณ อับดับเมือง
merge new_df and daily_df

In [20]:
daily_station = pd.merge(daily_df, new_df,on="station_id")

In [21]:
daily_station.sort_values(by=["year", "month"], inplace = True)

In [22]:
daily_station

,datetime,month,year,station_id,PM2.5,PM10,NOx,O3,CO,HR,NO,NO2,TMP,network_name
0,2010-01-20,1,2010,68,NaN,50.444444,0.019147,NaN,NaN,35.205704,0.007604,0.011870,22.005545,Celaya
1,2010-01-21,1,2010,68,NaN,67.500000,0.042956,NaN,NaN,42.276420,0.028849,0.014479,16.882448,Celaya
2,2010-01-22,1,2010,68,NaN,67.291667,NaN,NaN,NaN,NaN,NaN,0.015071,NaN,Celaya
3,2010-01-23,1,2010,68,NaN,69.500000,NaN,NaN,NaN,NaN,NaN,0.014679,NaN,Celaya
4,2010-01-24,1,2010,68,NaN,47.500000,NaN,NaN,NaN,NaN,NaN,0.013279,NaN,Celaya
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226590,2021-12-27,12,2021,426,28.956522,67.375000,0.036429,0.030750,1.684583,56.541667,NaN,0.024762,21.660833,Monterrey
226591,2021-12-28,12,2021,426,20.818182,55.833333,0.037508,0.026792,1.700000,44.375000,NaN,0.023225,23.731250,Monterrey
226592,2021-12-29,12,2021,426,16.857143,51.458333,0.045588,0.017083,1.732917,37.875000,NaN,0.026500,22.751250,Monterrey
226593,2021-12-30,12,2021,426,21.555556,51.750000,0.037229,0.029833,1.520000,42.916667,NaN,0.023929,22.988333,Monterrey
